In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

In [2]:
from micrograd.engine import Value
from micrograd.nn import MLP

In [3]:
sys.setrecursionlimit(10000)

In [4]:
iris = load_iris()

X = np.array(iris.data)
y = np.array(iris.target)

ratio_sets = 0.8
size = int(150 * ratio_sets)

scaler = StandardScaler()
X = scaler.fit_transform(X)

r_index= list(np.random.permutation(len(y)))

X_train, y_train = X[r_index[:size]], y[r_index[:size]]
X_test, y_test = X[r_index[size:]], y[r_index[size:]]

print("Dataset dimension:")

print("Training set = ", len(X_train), len(y_train))
print("Test set = ", len(X_test), len(y_test))

print("Dimension of input = ", len(X_train[0]))

Dataset dimension:
Training set =  120 120
Test set =  30 30
Dimension of input =  4


In [5]:
print("Feature names:", iris.feature_names)
print("Target names:", iris.target_names)

print("First 5 samples:")
for i in range(5):
    print(f"Sample {i+1}: {X[i]} (Class: {y[i]}, Species: {iris.target_names[y[i]]})")

Feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
Target names: ['setosa' 'versicolor' 'virginica']
First 5 samples:
Sample 1: [-0.90068117  1.01900435 -1.34022653 -1.3154443 ] (Class: 0, Species: setosa)
Sample 2: [-1.14301691 -0.13197948 -1.34022653 -1.3154443 ] (Class: 0, Species: setosa)
Sample 3: [-1.38535265  0.32841405 -1.39706395 -1.3154443 ] (Class: 0, Species: setosa)
Sample 4: [-1.50652052  0.09821729 -1.2833891  -1.3154443 ] (Class: 0, Species: setosa)
Sample 5: [-1.02184904  1.24920112 -1.34022653 -1.3154443 ] (Class: 0, Species: setosa)


In [6]:
model = MLP(4, [5, 3])
print('number of parameters', len(model.parameters()))
print(model)

number of parameters 43
MLP of [Layer of [ReLu Neuron (4), ReLu Neuron (4), ReLu Neuron (4), ReLu Neuron (4), ReLu Neuron (4)], Layer of [Softmax Neuron (5), Softmax Neuron (5), Softmax Neuron (5)]]


In [7]:
# loss function
def loss():
    inputs = [list(map(Value, xrow)) for xrow in X_train]
    
    # forward the model to get scores
    scores = list(map(model, inputs)) 
    
    # Cross-entropy Loss
    losses = [(- score[yi].log()) for yi, score in zip(y_train, scores)]
    total_loss = sum(losses) / len(losses)

    # Accuracy
    accuracy = []
    for yi, score in zip(y_train, scores):
        argmax = -1
        valmax = 0
        for idx, scorei in enumerate(score):
            if scorei.data > valmax:
                valmax = scorei.data
                argmax = idx
        accuracy.append((argmax == yi))

    acc = sum(accuracy) / len(accuracy)
        
    return total_loss, acc

total_loss, acc = loss()
print(f"step {0} loss {total_loss.data}, accuracy {acc*100}%")

step 0 loss 1.2986896141830822, accuracy 24.166666666666668%


In [8]:
# backward
for k in range(400):
    model.zero_grad()
    total_loss.backward()
        
    # update (sgd)
    learning_rate = 0.1
    for p in model.parameters():
        p.data -= learning_rate * p.grad

    total_loss, acc = loss()
    
    if k % 10 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")

step 0 loss 1.2436923680712217, accuracy 25.0%
step 10 loss 0.9642199278883227, accuracy 64.16666666666667%
step 20 loss 0.8158705671109199, accuracy 75.83333333333333%
step 30 loss 0.7093393116585764, accuracy 85.0%
step 40 loss 0.619341732195803, accuracy 92.5%
step 50 loss 0.53145548678982, accuracy 94.16666666666667%
step 60 loss 0.44301416738477334, accuracy 95.0%
step 70 loss 0.36822854111973863, accuracy 95.83333333333334%
step 80 loss 0.31369323343543337, accuracy 95.83333333333334%
step 90 loss 0.2755089199534364, accuracy 95.83333333333334%
step 100 loss 0.24755164746623867, accuracy 96.66666666666667%
step 110 loss 0.22596287353358785, accuracy 96.66666666666667%
step 120 loss 0.20859950313554004, accuracy 96.66666666666667%
step 130 loss 0.19409198193164945, accuracy 96.66666666666667%
step 140 loss 0.18169474835561492, accuracy 96.66666666666667%
step 150 loss 0.1708734606726693, accuracy 96.66666666666667%
step 160 loss 0.16131724700398994, accuracy 95.83333333333334%
ste

In [9]:
def test(batch_size=None):
    inputs = [list(map(Value, xrow)) for xrow in X_test]
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # Cross-entropy Loss
    losses = [(- score[yi].log()) for yi, score in zip(y_test, scores)]
    total_loss = sum(losses) / len(losses)

    # Accuracy
    predict = []
    accuracy = []
    for yi, score in zip(y_test, scores):
        argmax = -1
        valmax = 0
        for idx, scorei in enumerate(score):
            if scorei.data > valmax:
                valmax = scorei.data
                argmax = idx
        predict.append((argmax, yi))
        accuracy.append((argmax == yi))

    acc = sum(accuracy) / len(accuracy)
        
    return total_loss, acc, predict

In [10]:
# evaluate
total_loss, acc, pred = test()
print(f"Loss = {total_loss.data}, accuracy = {acc*100}%")

Loss = 0.11123970982618364, accuracy = 96.66666666666667%
